In [2]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import latex2markdown

from tools_cv import *
from tools_kinematics import *

In [10]:
text = 'latex'
# text = 'markdown'
def write_text(left, alias=None, right=None, ans=None, simplify=True):
    def _concat_expression(expression):
        if type(expression) is list:
            exp = ''
            for each in expression:
                if simplify:
                    each = sp.simplify(each)
                exp = exp + '{' + sp.latex(each.subs(alias)) + '}'
        else:
            if simplify:
                expression = sp.simplify(expression)
            exp = sp.latex(expression.subs(alias))
        return exp

    if left is not None:
        left = _concat_expression(left)
    if right is not None:
        right = _concat_expression(right)

    if right is None:
        expr_latex = left + '=' + str(ans)
    elif ans is None:
        expr_latex = left + '=' + right
    else:
        expr_latex = left + '=' + right + '=' + str(ans)
        
    expr = expr_latex.replace(r'\\', r'\\' + '\n')
    if text == 'markdown':
        expr = '```math\n' + expr_latex + '\n```'
    print(expr)
    print()
    

In [18]:
# Constants
A = get_sym('A')  # car length
B = get_sym('B')  # car width / track
C = get_sym('C')  # car height
l = get_sym('l')  # wheelbase
r = get_sym('r')  # wheel radius
R = get_sym('R')  # turning radius (Ackerman)

X = get_sym(r'X')
X_dot = get_sym(r'X', dot=1)

Y = get_sym(r'Y')
Y_dot = get_sym(r'Y', dot=1)

Z = get_sym(r'Z')
Z_dot = get_sym(r'Z', dot=1)

theta = get_sym(r'\theta')
theta_dot = get_sym(r'\theta', dot=1)

alias = {}

# display cos(theta) as c\theta, sin(theta) as s\theta
alias.update({
    sp.sin(theta): sp.symbols(r"s\theta"),
    sp.cos(theta): sp.symbols(r"c\theta"),
})

In [19]:
wheels = ['FL', 'FR', 'RL', 'RR']
L_FL = L_FR = B / 2
L_RL = L_RR = sp.sqrt(B**2 / 4 + l**2)
all_L = [L_FL, L_FR, L_RL, L_RR]

for wheel in wheels:
    alpha = get_sym(r'\alpha', wheel)
    exec(f"alpha_{wheel} = alpha")
    
    beta = get_sym(r'\beta', wheel)
    exec(f"beta_{wheel} = beta")
    
    phi = get_sym(r'\phi', wheel, dot=1)
    exec(f"phi_{wheel} = phi")
    
    # display cos(beta) as c\theta, sin(beta) as s\theta
    alias.update({
        sp.sin(beta): sp.symbols(r"s\beta_{%s}" % wheel),
        sp.cos(beta): sp.symbols(r"c\beta_{%s}" % wheel),
    })

In [20]:
# Overwrite constants
alpha_FL = sp.pi/2
alpha_FR = -sp.pi/2
alpha_RL = sp.pi/2 + sp.atan(2*l/B)
alpha_RR = -sp.pi/2 - sp.atan(2*l/B)

beta_RL = -sp.atan(2*l/B)
beta_RR = sp.atan(2*l/B) + sp.pi

In [21]:
write_text(get_sym('L', 'FL'), alias, L_FL)
write_text(get_sym('L', 'RL'), alias, L_RL)
write_text(get_sym(r'\alpha', 'FL'), alias, alpha_FL)
write_text(get_sym(r'\alpha', 'FR'), alias, alpha_FR)
write_text(get_sym(r'\alpha', 'RL'), alias, alpha_RL)
write_text(get_sym(r'\alpha', 'RR'), alias, alpha_RR)
write_text(get_sym(r'\beta', 'RL'), alias, beta_RL)
write_text(get_sym(r'\beta', 'RR'), alias, beta_RR)

L_{FL}=\frac{B}{2}

L_{RL}=\frac{\sqrt{B^{2} + 4 l^{2}}}{2}

\alpha_{FL}=\frac{\pi}{2}

\alpha_{FR}=- \frac{\pi}{2}

\alpha_{RL}=\operatorname{atan}{\left(\frac{2 l}{B} \right)} + \frac{\pi}{2}

\alpha_{RR}=- \operatorname{atan}{\left(\frac{2 l}{B} \right)} - \frac{\pi}{2}

\beta_{RL}=- \operatorname{atan}{\left(\frac{2 l}{B} \right)}

\beta_{RR}=\operatorname{atan}{\left(\frac{2 l}{B} \right)} + \pi



In [22]:
RI0 = rotation(theta, 'z')
R0I = RI0.T
R0I_sym = get_sym('R', 'I', '0')
RI0_sym = get_sym('R', '0', 'I')
eqnprint(R0I_sym, alias, R0I)

P_dot = sp.Matrix([X_dot, Y_dot, theta_dot])
P_sym = get_sym('P', '', 'I', dot=1)
eqnprint(P_sym, alias, P_dot)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [30]:
# rolling condition
M_rolling_FL = get_mat_rolling(alpha_FL, beta_FL, L_FL)
M_rolling_FR = get_mat_rolling(alpha_FR, beta_FR, L_FR)
M_rolling_RL = get_mat_rolling(alpha_RL, beta_RL, L_RL)
M_rolling_RR = get_mat_rolling(alpha_RR, beta_RR, L_RR)

M_rolling = sp.Matrix([M_rolling_FL, M_rolling_FR,
                       M_rolling_RL, M_rolling_RR])

# so sliding condition
M_no_sliding_FL = get_mat_no_sliding(alpha_FL, beta_FL, L_FL)
M_no_sliding_FR = get_mat_no_sliding(alpha_FR, beta_FR, L_FR)
M_no_sliding_RL = get_mat_no_sliding(alpha_RL, beta_RL, L_RL)
M_no_sliding_RR = get_mat_no_sliding(alpha_RR, beta_RR, L_RR)

M_no_sliding = sp.Matrix([M_no_sliding_FL, M_no_sliding_FR,
                          M_no_sliding_RL, M_no_sliding_RR])

print('Rolling condition:')
left = [M_rolling,
        R0I_sym,
        P_sym,
        get_sym('-'),
        get_sym('r'),
        sp.Matrix([phi_FL, phi_FR, phi_RL, phi_RR])]
eqnprint(left, alias, None , 0)
write_text(left, alias, None , 0)

print('No-sliding condition:')
left = [M_no_sliding,
        R0I_sym,
        P_sym,
        ]
eqnprint(left, alias, None , 0)
write_text(left, alias, None , 0)


Rolling condition:


<IPython.core.display.Math object>

{\left[\begin{matrix}c\beta_{FL} & s\beta_{FL} & - \frac{B c\beta_{FL}}{2}\\
- c\beta_{FR} & - s\beta_{FR} & - \frac{B c\beta_{FR}}{2}\\
1 & 0 & - \frac{\sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
1 & 0 & \frac{\sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]}{^{0}R_{I}}{\dot{^IP}}{-}{r}{\left[\begin{matrix}\dot{\phi_{FL}}\\
\dot{\phi_{FR}}\\
\dot{\phi_{RL}}\\
\dot{\phi_{RR}}\end{matrix}\right]}=0

No-sliding condition:


<IPython.core.display.Math object>

{\left[\begin{matrix}- s\beta_{FL} & c\beta_{FL} & \frac{B s\beta_{FL}}{2}\\
s\beta_{FR} & - c\beta_{FR} & \frac{B s\beta_{FR}}{2}\\
0 & 1 & - \frac{l \sqrt{B^{2} + 4 l^{2}}}{B \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
0 & 1 & - \frac{l \sqrt{B^{2} + 4 l^{2}}}{B \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]}{^{0}R_{I}}{\dot{^IP}}=0



In [24]:
eqnprint(get_sym('M', 'rollingFL'), alias, M_rolling_FL)
write_text(get_sym('M', 'rollingFL'), alias, M_rolling_FL)

eqnprint(get_sym('M', 'rollingFR'), alias, M_rolling_FR)
write_text(get_sym('M', 'rollingFR'), alias, M_rolling_FR)

eqnprint(get_sym('M', 'rollingRL'), alias, M_rolling_RL)
write_text(get_sym('M', 'rollingRL'), alias, M_rolling_RL)

eqnprint(get_sym('M', 'rollingRR'), alias, M_rolling_RR)
write_text(get_sym('M', 'rollingRR'), alias, M_rolling_RR)


eqnprint(get_sym('M', 'no_slidingFL'), alias, M_no_sliding_FL)
write_text(get_sym('M', 'no_slidingFL'), alias, M_no_sliding_FL)

eqnprint(get_sym('M', 'no_slidingFR'), alias, M_no_sliding_FR)
write_text(get_sym('M', 'no_slidingFR'), alias, M_no_sliding_FR)

eqnprint(get_sym('M', 'no_slidingRL'), alias, M_no_sliding_RL)
write_text(get_sym('M', 'no_slidingRL'), alias, M_no_sliding_RL)

eqnprint(get_sym('M', 'no_slidingRR'), alias, M_no_sliding_RR)
write_text(get_sym('M', 'no_slidingRR'), alias, M_no_sliding_RR)

<IPython.core.display.Math object>

M_{rollingFL}=\left[\begin{matrix}c\beta_{FL} & s\beta_{FL} & - \frac{B c\beta_{FL}}{2}\end{matrix}\right]



<IPython.core.display.Math object>

M_{rollingFR}=\left[\begin{matrix}- c\beta_{FR} & - s\beta_{FR} & - \frac{B c\beta_{FR}}{2}\end{matrix}\right]



<IPython.core.display.Math object>

...

M_{no_slidingFR}=\left[\begin{matrix}s\beta_{FR} & - c\beta_{FR} & \frac{B s\beta_{FR}}{2}\end{matrix}\right]



<IPython.core.display.Math object>

M_{no_slidingRL}=\left[\begin{matrix}0 & 1 & - \frac{l \sqrt{B^{2} + 4 l^{2}}}{B \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]



<IPython.core.display.Math object>

M_{no_slidingRR}=\left[\begin{matrix}0 & 1 & - \frac{l \sqrt{B^{2} + 4 l^{2}}}{B \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]



In [25]:
M_angle = sp.Matrix([M_rolling_RL, M_rolling_RR, M_no_sliding_RL])
M_r = sp.eye(2)*r
M_r = M_r.row_insert(len(M_r), sp.zeros(1, 2))
M_phi = sp.Matrix([phi_RL, phi_RR])


left = [M_angle,
        R0I_sym,
        P_sym,
        get_sym('-'),
        M_r,
        M_phi]
eqnprint(left, alias, None , 0)
write_text(left, alias, None , 0)

<IPython.core.display.Math object>

{\left[\begin{matrix}1 & 0 & - \frac{\sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
1 & 0 & \frac{\sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
0 & 1 & - \frac{l \sqrt{B^{2} + 4 l^{2}}}{B \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]}{^{0}R_{I}}{\dot{^IP}}{-}{\left[\begin{matrix}r & 0\\
0 & r\\
0 & 0\end{matrix}\right]}{\left[\begin{matrix}\dot{\phi_{RL}}\\
\dot{\phi_{RR}}\end{matrix}\right]}=0



In [26]:
M_temp = M_angle.inv() * M_r
eqnprint(P_sym, alias, [RI0_sym,
                        M_temp,
                        M_phi])
write_text(P_sym, alias, [RI0_sym,
                        M_temp,
                        M_phi])

M_temp = M_temp * M_phi
eqnprint(P_sym, alias, [RI0_sym,
                        M_temp])
write_text(P_sym, alias, [RI0_sym,
                        M_temp])

P_I_dot = RI0 * M_temp
eqnprint(P_sym, alias, P_I_dot)
write_text(P_sym, alias, P_I_dot)

<IPython.core.display.Math object>

\dot{^IP}={^{I}R_{0}}{\left[\begin{matrix}\frac{r}{2} & \frac{r}{2}\\
- \frac{l r}{B} & \frac{l r}{B}\\
- \frac{r \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}{\sqrt{B^{2} + 4 l^{2}}} & \frac{r \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}{\sqrt{B^{2} + 4 l^{2}}}\end{matrix}\right]}{\left[\begin{matrix}\dot{\phi_{RL}}\\
\dot{\phi_{RR}}\end{matrix}\right]}



<IPython.core.display.Math object>

\dot{^IP}={^{I}R_{0}}{\left[\begin{matrix}\frac{r \left(\dot{\phi_{RL}} + \dot{\phi_{RR}}\right)}{2}\\
\frac{l r \left(- \dot{\phi_{RL}} + \dot{\phi_{RR}}\right)}{B}\\
\frac{r \sqrt{\frac{B^{2} + 4 l^{2}}{B^{2}}} \left(- \dot{\phi_{RL}} + \dot{\phi_{RR}}\right)}{\sqrt{B^{2} + 4 l^{2}}}\end{matrix}\right]}



<IPython.core.display.Math object>

\dot{^IP}=\left[\begin{matrix}\frac{r \left(B c\theta \left(\dot{\phi_{RL}} + \dot{\phi_{RR}}\right) + 2 l s\theta \left(\dot{\phi_{RL}} - \dot{\phi_{RR}}\right)\right)}{2 B}\\
\frac{r \left(B s\theta \left(\dot{\phi_{RL}} + \dot{\phi_{RR}}\right) - 2 c\theta l \left(\dot{\phi_{RL}} - \dot{\phi_{RR}}\right)\right)}{2 B}\\
\frac{r \sqrt{\frac{B^{2} + 4 l^{2}}{B^{2}}} \left(- \dot{\phi_{RL}} + \dot{\phi_{RR}}\right)}{\sqrt{B^{2} + 4 l^{2}}}\end{matrix}\right]



## Inverse

In [34]:
M_temp_left = M_r.T.multiply(M_r)

M_temp_right = M_r.T.multiply(M_angle)
matprint(sp.simplify(M_temp_right))
write_text(get_sym('a', 'a'), alias, M_temp_right)

M_temp = M_temp_left.inv().multiply(M_temp_right)
matprint(sp.simplify(M_temp))
write_text(get_sym('a', 'a'), alias, M_temp_right)

M_inverse = M_temp.multiply(R0I)
M_inverse = sp.simplify(M_inverse)
eqnprint(get_sym('M', 'inverse'), alias, M_inverse)
write_text(get_sym('M', 'inverse'), alias, M_inverse)

Matrix([
[r, 0, -r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))],
[r, 0,  r*sqrt(B**2 + 4*l**2)/(2*sqrt(1 + 4*l**2/B**2))]])

a_{a}=\left[\begin{matrix}r & 0 & - \frac{r \sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
r & 0 & \frac{r \sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]



Matrix([
[1/r, 0, -sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))],
[1/r, 0,  sqrt(B**2 + 4*l**2)/(2*r*sqrt(1 + 4*l**2/B**2))]])

a_{a}=\left[\begin{matrix}r & 0 & - \frac{r \sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
r & 0 & \frac{r \sqrt{B^{2} + 4 l^{2}}}{2 \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]



<IPython.core.display.Math object>

M_{inverse}=\left[\begin{matrix}\frac{c\theta}{r} & \frac{s\theta}{r} & - \frac{\sqrt{B^{2} + 4 l^{2}}}{2 r \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\\
\frac{c\theta}{r} & \frac{s\theta}{r} & \frac{\sqrt{B^{2} + 4 l^{2}}}{2 r \sqrt{1 + \frac{4 l^{2}}{B^{2}}}}\end{matrix}\right]

